In [1]:
import requests

In [2]:
# try look at json data without using the package (save u the hassle of using asyncio)
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# h2h league
league_code = 982695 # rbsc h2h
gw_no = 10

## pulling data from h2h league

In [3]:
def get_h2h_gw_scores(league_code:int, gw_no:int) -> dict:
    '''Get h2h league gw score'''
    # get h2h league data
    r = requests.get(base_url+'leagues-h2h-matches/league/'+str(league_code)+'/?event='+str(gw_no))
    assert r.status_code == 200, r.status_code

    # parse data
    data = {
        'team_id':[],
        'team_names':[],
        'player_names':[],
        'points_h2h':[],
    }

    results = r.json()['results'] # r.json()['results'] is a list of dictionary, each element represents a fixture within the h2h league
    for result in results:
        # odd number of players so one fixture will face an average, don't include that
        if result['entry_1_name'] != 'AVERAGE':
            data['team_id'].append(result['entry_1_entry'])
            data['team_names'].append(result['entry_1_name'])
            data['player_names'].append(result['entry_1_player_name'])
            data['points_h2h'].append(result['entry_1_points'])
        if result['entry_2_name'] != 'AVERAGE':
            data['team_id'].append(result['entry_2_entry'])
            data['team_names'].append(result['entry_2_name'])
            data['player_names'].append(result['entry_2_player_name'])
            data['points_h2h'].append(result['entry_2_points'])

    assert len(data['team_names']) == 55 # ! hard code ไปก่อน

    return data

In [4]:
data = get_h2h_gw_scores(league_code, gw_no)

In [5]:
data.keys()

dict_keys(['team_id', 'team_names', 'player_names', 'points_h2h'])

## pulling data from team id view and not referring to h2h league also works
- this gives you lots of data to play with

In [6]:
def get_team(team_id:int, gw_no:int) -> dict:
    '''get team data for a specific gameweek'''
    r = requests.get(base_url+'entry/'+str(team_id)+'/event/'+str(gw_no)+'/picks/')
    assert r.status_code == 200
    return r.json()

def get_team_points_classic(team_id:int, gw_no:int) -> int:
    '''get classic league gw points from each team this gw'''
    return get_team(team_id, gw_no)['entry_history']['points']

def get_team_transfers_cost(team_id:int, gw_no:int) -> int:
    '''get transfers point penalty from each team this gw'''
    return get_team(team_id, gw_no)['entry_history']['event_transfers_cost']

In [7]:
data['points_classic'] = []
for team_id in data['team_id']:
    data['points_classic'].append(get_team_points_classic(team_id, gw_no))

In [8]:
data['transfers_cost'] = []
for team_id in data['team_id']:
    data['transfers_cost'].append(get_team_transfers_cost(team_id, gw_no))

In [9]:
data.keys()

dict_keys(['team_id', 'team_names', 'player_names', 'points_h2h', 'points_classic', 'transfers_cost'])

## calculate weekly summary

In [10]:
# find top and bottom
max_indices = [index for index, item in enumerate(data['points_h2h']) if item == max(data['points_h2h'])]
min_indices = [index for index, item in enumerate(data['points_h2h']) if item == min(data['points_h2h'])]

In [11]:
# payment details
prize_money = 500

if len(max_indices) == 1 and len(min_indices) == 1:
    # champ คนเดียว บ๊วยคนเดียว
    payees = [prize_money]
    payers = [-1 * prize_money]
elif len(max_indices) == 1 and len(min_indices) > 1:
    # champ คนเดียว บ๊วยหลายคน, บ๊วยจ่ายทุกคน
    payees = [prize_money * len(min_indices)] 
    payers = [-1 * prize_money] * len(min_indices)
elif len(max_indices) > 1 and len(min_indices) == 1:
    # champ หลายคน บ๊วยคนเดียว, แชมป์หารรับ
    payees = [round(prize_money / len(max_indices), 2)] * len(max_indices)
    payers = [-1 * prize_money] 
else:
    # champ หลายคน บ๊วยหลายคน
    payees = [round(prize_money * len(min_indices) / len(max_indices), 2)] * len(max_indices)
    payers = [-1 * prize_money] * len(min_indices) 

In [12]:
# result message
print('GW', gw_no, 'H2H results')

print('\nTop scorer(s):')
for max_index in max_indices:
    print(
        data['team_names'][max_index],
        '('+data['player_names'][max_index]+')',
        data['points_classic'][max_index], '-', data['transfers_cost'][max_index], # redundant, checking math logic of classic and h2h leagues
        '=', data['points_h2h'][max_index], 'Points'
        )

print('\nBottom(s) of the week:')
for min_index in min_indices:
    print(
        data['team_names'][min_index],
        '('+data['player_names'][min_index]+')',
        data['points_classic'][min_index], '-', data['transfers_cost'][min_index],
        '=', data['points_h2h'][min_index], 'Points'
        )

print()
print(payees)
print(payers)

GW 10 H2H results

Top scorer(s):
peep (Peep Chutrakul) 89 - 0 = 89 Points

Bottom(s) of the week:
PIYAWIT (Piyawit TAC) 38 - 0 = 38 Points

[500]
[-500]
